## **Configuring the account**

In [ ]:
import subprocess, os
from dotenv import load_dotenv
# identifying into github
!git config --global user.name "Nuzz23"
!git config --global user.email "nunzio.licalzi9@gmail.com"
output = subprocess.check_output("git config --global --list", shell=True).decode('utf-8').split()

# check if correctly identified
assert len(output) >= 2, "Wrong lenght"
assert output[0].split('=')[-1] == 'Nuzz23', 'wrong user name'
assert output[1].split('=')[-1] == 'nunzio.licalzi9@gmail.com', 'wrong email'
del output

load_dotenv()

In [ ]:
with open("TOKENS.txt", 'r', encoding='utf-8') as f:
  for line in f:
    os.environ[line.split('=')[0].strip()] = line.strip().split('=')[1].strip()

## **Cloning the repository**

In [ ]:
# if the cloned repository already exists delete it
%cd /content
%rm -rf sample_data
if "MLDL_SemanticSegmentation" in subprocess.check_output("ls", shell=True).decode("utf-8").strip():
  !rm -rf MLDL_SemanticSegmentation

/content


In [ ]:
# clone the repository via token
!git clone --quiet "{os.getenv('GITHUB_TOKEN')}"

# check if cloned correctly
assert "MLDL_SemanticSegmentation" in subprocess.check_output("ls", shell=True).decode("utf-8").strip(), "Not cloned correctly"

/content/MLDL_SemanticSegmentation


# **LIBRARIES**

In [ ]:
# installing the required libraries
%cd /content/MLDL_SemanticSegmentation
%pip install -q -r requirements.txt

## **MAIN LIBRARIES**

In [ ]:
import torch, wandb

## **CUSTOM IMPORTS**

In [ ]:
from stats import countFLOPS, latency, evaluateLastEpoch
from train.trainBiSeNetCity import initBiSeNetOrV2Base
from datasets.downloader import Downloader

# **DATASET**

## **Downloading the dataset**

In [ ]:
if not Downloader().downloadCityScapes():
  raise FileNotFoundError("CityScapes dataset not found")

# **Model**

In [ ]:
WIDTH, HEIGHT = 1024, 512

model = initBiSeNetOrV2Base(model_str='bisenet', batchSize=4, learning_rate=0.0005, width=WIDTH, height=HEIGHT,
                    pushWeights=True, enablePrintVal=True, enablePrint=True, restartTraining=True)

In [ ]:
print('flops', countFLOPS(model, width=WIDTH, height=HEIGHT))

print('latencyMean, latencyStd, fpMean, fpsStd', *latency(model, width=WIDTH, height=HEIGHT))

print(f"Numero totale di parametri: {sum(p.numel() for p in model.parameters())}")

evaluateLastEpoch(model)

context_path.features.avgpool, context_path.features.fc, supervision1, supervision2


flops 25779775698
latencyMean, latencyStd, fpMean, fpsStd 15.839773893356325 2.301098004268971 64.58737348178171 13.600086919463912
Numero totale di parametri: 12581672
